1. Парсинг hh.ru

In [ ]:
from IPython.display import HTML, display
import requests
import time
from bs4 import BeautifulSoup
import json
import tqdm
from google.colab import files

# для многострочного вывода текста в collab
# было нужно для удобства отладки на первых этапах
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# подменяем заголовок
headers = {
    'authority': 'hh.ru',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',    
    'sec-ch-ua': '"Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"',
    'sec-ch-ua-mobile': '?1',
    'sec-ch-ua-platform': '"Android"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36',
}

# парамеры поиска
params = {
    'experience': 'between1And3',
    'schedule': 'fullDay',    
    'text': 'python разработчик',
    'no_magic': 'true',  
    'page': '0',
    'items_on_page': '20',
}

# список дополнительных параметров поиска для обхода ограничения hh на выдачу не более 2000 вакансий за раз
# при необходимости можно дополнить ещё парой списков и перебирать и по ним тоже
exp_list = ['noExperience', 'between1And3', 'between3And6', 'moreThan6']
sched_list = ['fullDay', 'remote', 'flexible', 'shift']
# просто чтобы не делать два цикла
iter_list = [[l1, l2] for l1 in exp_list for l2 in sched_list]

# переменная для сохрания результата
data = {
    'data':[]
}


for exp_val, sched_val in tqdm.tqdm(iter_list):
  params['experience'] = exp_val  
  params['schedule'] = sched_val
  # перебираем с 0 до 99 страницы
  for page in range(0, 100):
    params['page'] = str(page)
    # запрос для первичного поиска
    with requests.get('https://hh.ru/search/vacancy', params=params, headers=headers) as r:      
      root_data = BeautifulSoup(r.content, 'lxml')
      vacancies = root_data.find_all('div', attrs={"class":"vacancy-serp-item-body__main-info"})
      # проверяем не пустой ли список вакансий. он может быть пустым, если вакансий меньше чем максимальное количество страниц
      if vacancies:
        for vacancy in vacancies:
          title = vacancy.find('a', attrs={'class':'serp-item__title'})
          href = title['href']
          address = vacancy.find('div',  attrs={'data-qa':'vacancy-serp__vacancy-address'})
          # регион идёт в начале адреса, потом может идти доп. информация о станциях метро и пр.
          region = [i for i in address.strings][0]
          salary = vacancy.find('span',  attrs={'data-qa':'vacancy-serp__vacancy-compensation'})        
          if salary:
            # убираем неразрыный пробел. он в общем, не мешает, но некрасиво показывается в текстовых редакторах
            salary = salary.text.replace('\u202f', '')
          
          # ждём немного, чтобы нас не забанили за ddos
          time.sleep(0.5)
          # открываем вакансию по ссылке. внутри нам нужна информация о стаже
          with requests.get(href, headers=headers) as v:
            vacancy_data = BeautifulSoup(v.content, 'lxml')
            experience = vacancy_data.find('span', attrs={'data-qa':'vacancy-experience'})
            data['data'].append({'title':title.string if title else None, 'work experience':experience.string if experience else None, 'salary':salary, 'region':region})            
      # если не нашли вакансий на странице, то значит прерываем текущую итерацию
      else:
        break                
  
  

with open('data.json', 'w') as f:
  json.dump(data, f, ensure_ascii=False)

files.download('data.json')

2. Палиндром строки

In [5]:
s = str(input('Введите строку ')).replace(' ', '')
print(s == s[::-1])

Введите строку black cat
tackcalb
False


3. Перевод арабского числа в римское

In [27]:
def arabic_to_roman(n):
  digits = {1000:'M', 900:'CM', 500:'D', 400:'CD', 100:'C', 90:'XC', 50:'L', 40:'XL', 10:'X', 9:'IX', 5:'V', 4:'IV', 1:'I'}
  result = ''
  for arabic, roman in digits.items():
    result += n // arabic * roman
    n %= arabic        
  return result

print(arabic_to_roman(int(input())))

113
CXIII


4. Валидность скобок

In [28]:
def check_brackets(s):
  # допускается использование любых сиволов помимо скобок, 
  # легко дополнить "словарь" скобок, добавив новые элементы в b_open и b_close
  b_open = ('(', '[', '{')
  b_close = (')', ']', '}')
  stack = []
  for char in s:
    if char in b_open:
      stack.append(char)
    elif char in b_close:
      open_char = b_open[b_close.index(char)]
      if len(stack) > 0 and open_char == stack[-1]:
        stack.pop(-1)
      else:
        return False
  if len(stack) == 0:
    return True
  else: 
    return False

print(check_brackets(str(input())))

[1{2}3(4{}443)3]
True


5. Умножить два бинарных числа в формате строк

In [31]:
# В условии задачи не задано ограничение на использование преобразования типов и побитовых операций
# Поэтому не будем умножать в столбик

x1 = str(input())
x2 = str(input())
 
x1 = int(x1, 2)
x2 = int(x2, 2)
 
x = x1 * x2

# используем срез, чтобы убрать 0b в начале строки
print(str(bin(x))[2:])

111
101
100011
